# matala 3 

In [1]:
import pandas as pd
import urllib.request
import json

## # 1

In [2]:
# Replace with your own API key
api_key = 'Replace with your own API key'

# Read destinations from file
with open('dests.txt', 'r') as f:
    destinations = f.read().splitlines()

# Initialize empty lists to store data
targets = []
distances = []
durations = []
longitudes = []
latitudes = []

# Loop over destinations
for destination in destinations:
    try:
        # Encode destination for use in URL
        destination_encoded = urllib.parse.quote(destination)

        # Distance Matrix API request
        distance_matrix_url = f'https://maps.googleapis.com/maps/api/distancematrix/json?origins=Tel+Aviv&destinations={destination_encoded}&key={api_key}'
        with urllib.request.urlopen(distance_matrix_url) as response:
            data = json.load(response)
            row = data['rows'][0]['elements'][0]
            distance_km = row['distance']['value'] / 1000
            duration_min = row['duration']['value'] / 60

        # Geocoding API request
        geocode_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={destination_encoded}&key={api_key}'
        with urllib.request.urlopen(geocode_url) as response:
            data = json.load(response)
            location = data['results'][0]['geometry']['location']
            longitude = location['lng']
            latitude = location['lat']

        # Append data to lists
        targets.append(destination)
        distances.append(distance_km)
        durations.append(duration_min)
        longitudes.append(longitude)
        latitudes.append(latitude)

    except Exception as e:
        print(f'Error occurred for {destination}')
        print(f'Error message: {str(e)}')

# Create DataFrame from the lists
df = pd.DataFrame({
    'Target': targets,
    'Distance km': distances,
    'Duration (hour+minutes)': durations,
    'Longitude': longitudes,
    'Latitude': latitudes
})

# Convert Duration (hour+minutes) to hh:mm format
df['Duration (hour+minutes)'] = df['Duration (hour+minutes)'].apply(lambda x: f'{int(x // 60):02d}:{int(x % 60):02d}')


## # 2

In [3]:
#DataFrame
df

,Target,Distance km,Duration (hour+minutes),Longitude,Latitude
0,Istanbul,1815.227,21:02,28.978359,41.008238
1,Amsterdam,4532.541,48:03,4.904139,52.367573
2,Valletta,3792.881,50:52,14.514100,35.899237
3,Basel,4092.805,44:02,7.588576,47.559599
4,Doha,2164.477,22:38,51.531040,25.285447


## #3

In [4]:
df1=df.nlargest(3, 'Distance km')
df1[['Target', 'Distance km']]

,Target,Distance km
1,Amsterdam,4532.541
3,Basel,4092.805
2,Valletta,3792.881
